In [1]:
# Lab-06 Softmax Classification
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

In [4]:
z = torch.FloatTensor([1, 2, 3])

In [5]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [6]:
hypothesis.sum()

tensor(1.)

In [8]:
# Cross Entropy -> 두 확률분포가 있을 때, 얼마나 두 확률분포가 비슷한지를 알려주는 수치
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.1664, 0.1871, 0.1737, 0.2695, 0.2033],
        [0.2002, 0.1783, 0.2218, 0.1944, 0.2054],
        [0.1809, 0.2380, 0.2318, 0.1084, 0.2409]], grad_fn=<SoftmaxBackward0>)


In [9]:
y = torch.randint(5, (3,)).long() 
print(y)

# Classes = 5 / samples = 3

tensor([3, 1, 2])


In [14]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # _(under bar)는 inplace 연산을 의미함. 첫번째 연산은 dim=1방향, 가운데는 index, 마지막은 삽입하고 싶은 숫자를 의미

tensor([[0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [11]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4992, grad_fn=<MeanBackward0>)


In [15]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]], grad_fn=<LogBackward0>)

In [16]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]],
       grad_fn=<LogSoftmaxBackward0>)

In [17]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4992, grad_fn=<MeanBackward0>)

In [18]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y) # NLL은 Negative Log Likleyhood

tensor(1.4992, grad_fn=<NllLossBackward0>)

In [19]:
# More high level
F.cross_entropy(z, y)

tensor(1.4992, grad_fn=<NllLossBackward0>)

In [25]:
x_train = [[1, 2, 1 ,1 ], [2, 1, 3, 2], [3, 1, 3, 4], [ 4, 1, 5, 5], [1, 7, 5, 5], [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [28]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# Optimizer 설정
optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 10000
for epoch in range(nb_epochs+1):

    # Cost 계산(1)
    hypothesis = F.softmax(x_train.matmul(W)+b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    # Cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/10000 Cost: 1.098612
Epoch  100/10000 Cost: 0.901535
Epoch  200/10000 Cost: 0.839114
Epoch  300/10000 Cost: 0.807826
Epoch  400/10000 Cost: 0.788472
Epoch  500/10000 Cost: 0.774822
Epoch  600/10000 Cost: 0.764449
Epoch  700/10000 Cost: 0.756191
Epoch  800/10000 Cost: 0.749398
Epoch  900/10000 Cost: 0.743671
Epoch 1000/10000 Cost: 0.738749
Epoch 1100/10000 Cost: 0.734453
Epoch 1200/10000 Cost: 0.730657
Epoch 1300/10000 Cost: 0.727271
Epoch 1400/10000 Cost: 0.724223
Epoch 1500/10000 Cost: 0.721463
Epoch 1600/10000 Cost: 0.718948
Epoch 1700/10000 Cost: 0.716644
Epoch 1800/10000 Cost: 0.714526
Epoch 1900/10000 Cost: 0.712570
Epoch 2000/10000 Cost: 0.710759
Epoch 2100/10000 Cost: 0.709075
Epoch 2200/10000 Cost: 0.707507
Epoch 2300/10000 Cost: 0.706043
Epoch 2400/10000 Cost: 0.704672
Epoch 2500/10000 Cost: 0.703385
Epoch 2600/10000 Cost: 0.702177
Epoch 2700/10000 Cost: 0.701038
Epoch 2800/10000 Cost: 0.699964
Epoch 2900/10000 Cost: 0.698950
Epoch 3000/10000 Cost: 0.697990
Epoch 31